In [1]:
%load_ext autoreload
%autoreload 1
import os

from matplotlib import pyplot as plt
import obspy
import pandas as pd

from insight.rotate import polarization_azimuth, angle_err
from insight.util import TraceFile

# Parameters

In [2]:
DATA_DIR = "../data/"
network = "XB"
station = "ELYSE"
location = "02"
bandpass = {"freqmin": 0.1, "freqmax": 0.7}
before, after = 10, 120

## Read events back from disk

In [3]:
st_zne = {}
events = pd.read_parquet(os.path.join(DATA_DIR, "a_quality.parquet"))
for evt in events.index:
    trace_file = TraceFile(network, station, location, events.loc[evt].id)
    st_zne[evt] = obspy.read(os.path.join(DATA_DIR, "zne", str(trace_file)))
events

,id,region name,time,quality,mqs_azimuth,mqs_distance,est_p_arrival,M_w
earthquake name,,,,,,,,
S1222a,mqs2022isne,Aeolis Northeast,2022-05-04T23:23:07.710516Z,A,109,37.01401401,2022-05-04T23:27:45.192771Z,4.6
S0976a,mqs2021qpls,Coprates Northwest,2021-08-25T03:32:20.629953Z,A,101,146.2592593,2021-08-25T03:48:56.429772Z,4.3
S1000a,mqs2021sijy,Lunae Palus Northwest,2021-09-18T17:46:20.639751Z,A,55,128.2912913,2021-09-18T17:58:51.682539Z,4.1
S1094b,mqs2021zdzn,Diacria Southwest,2021-12-24T22:38:02.749173Z,A,40,59.65365365,2021-12-24T22:45:08.063325Z,4.0
S1133c,mqs2022cixp,Elysium Southeast,2022-02-03T08:04:36.469259Z,A,90,30.00650651,2022-02-03T08:08:24.753506Z,3.8
S0235b,mqs2019onhx,Elysium Southeast,2019-07-26T12:15:38.748239Z,A,258.0304683,28.74874872,2019-07-26T12:19:16.991001Z,3.7
S0173a,mqs2019jzqa,Elysium Southeast,2019-05-23T02:19:09.852879Z,A,91,30.00650651,2019-05-23T02:22:58.043698Z,3.7
S1048d,mqs2021vwbn,Elysium Southeast,2021-11-07T22:00:15.254098Z,A,100,30.18618619,2021-11-07T22:04:04.40886Z,3.6
S1022a,mqs2021tyvj,Elysium Northeast,2021-10-11T23:14:29.105382Z,A,63,30.72522523,2021-10-11T23:18:20.004721Z,3.6


## Bandpass & DeTrend

In [4]:
for evt in events.index:
    st_zne[evt].detrend()
    st_zne[evt].filter(type="bandpass", **bandpass)

## Determine Polarisation

In [5]:
pa = {}

for evt in events.index:
    pa[evt] = polarization_azimuth(st_zne[evt], offset=before)._asdict()

In [6]:
# events = pd.merge(events, pd.DataFrame(pa).T, left_index=True, right_index=True)
events = pd.concat((events, pd.DataFrame(pa).T))
events["angle_err"] = events.apply(lambda e: angle_err(e.azimuth, float(e.mqs_azimuth)), axis=1)
events

# # Plot
# ax = plt.subplot(111, polar=True)
# ax.plot([i["azimuth"] for i in pa.values()], [i["incident_angle"] for i in pa.values()], "ro")
# ax.set_theta_direction(-1)
# ax.set_theta_zero_location("N")

,id,region name,time,quality,mqs_azimuth,mqs_distance,est_p_arrival,M_w,azimuth,incident_angle,angle_err
S1222a,mqs2022isne,Aeolis Northeast,2022-05-04T23:23:07.710516Z,A,109,37.01401401,2022-05-04T23:27:45.192771Z,4.6,NaN,NaN,NaN
S0976a,mqs2021qpls,Coprates Northwest,2021-08-25T03:32:20.629953Z,A,101,146.2592593,2021-08-25T03:48:56.429772Z,4.3,NaN,NaN,NaN
S1000a,mqs2021sijy,Lunae Palus Northwest,2021-09-18T17:46:20.639751Z,A,55,128.2912913,2021-09-18T17:58:51.682539Z,4.1,NaN,NaN,NaN
S1094b,mqs2021zdzn,Diacria Southwest,2021-12-24T22:38:02.749173Z,A,40,59.65365365,2021-12-24T22:45:08.063325Z,4.0,NaN,NaN,NaN
S1133c,mqs2022cixp,Elysium Southeast,2022-02-03T08:04:36.469259Z,A,90,30.00650651,2022-02-03T08:08:24.753506Z,3.8,NaN,NaN,NaN
S0235b,mqs2019onhx,Elysium Southeast,2019-07-26T12:15:38.748239Z,A,258.0304683,28.74874872,2019-07-26T12:19:16.991001Z,3.7,NaN,NaN,NaN
S0173a,mqs2019jzqa,Elysium Southeast,2019-05-23T02:19:09.852879Z,A,91,30.00650651,2019-05-23T02:22:58.043698Z,3.7,NaN,NaN,NaN
S1048d,mqs2021vwbn,Elysium Southeast,2021-11-07T22:00:15.254098Z,A,100,30.18618619,2021-11-07T22:04:04.40886Z,3.6,NaN,NaN,NaN
S1022a,mqs2021tyvj,Elysium Northeast,2021-10-11T23:14:29.105382Z,A,63,30.72522523,2021-10-11T23:18:20.004721Z,3.6,NaN,NaN,NaN
S0820a,mqs2021fjzq,Aeolis Northeast,2021-03-18T14:51:33.869889Z,A,88,30.18618619,2021-03-18T14:54:19.835741Z,3.3,NaN,NaN,NaN


In [7]:
events

,id,region name,time,quality,mqs_azimuth,mqs_distance,est_p_arrival,M_w,azimuth,incident_angle,angle_err
S1222a,mqs2022isne,Aeolis Northeast,2022-05-04T23:23:07.710516Z,A,109,37.01401401,2022-05-04T23:27:45.192771Z,4.6,NaN,NaN,NaN
S0976a,mqs2021qpls,Coprates Northwest,2021-08-25T03:32:20.629953Z,A,101,146.2592593,2021-08-25T03:48:56.429772Z,4.3,NaN,NaN,NaN
S1000a,mqs2021sijy,Lunae Palus Northwest,2021-09-18T17:46:20.639751Z,A,55,128.2912913,2021-09-18T17:58:51.682539Z,4.1,NaN,NaN,NaN
S1094b,mqs2021zdzn,Diacria Southwest,2021-12-24T22:38:02.749173Z,A,40,59.65365365,2021-12-24T22:45:08.063325Z,4.0,NaN,NaN,NaN
S1133c,mqs2022cixp,Elysium Southeast,2022-02-03T08:04:36.469259Z,A,90,30.00650651,2022-02-03T08:08:24.753506Z,3.8,NaN,NaN,NaN
S0235b,mqs2019onhx,Elysium Southeast,2019-07-26T12:15:38.748239Z,A,258.0304683,28.74874872,2019-07-26T12:19:16.991001Z,3.7,NaN,NaN,NaN
S0173a,mqs2019jzqa,Elysium Southeast,2019-05-23T02:19:09.852879Z,A,91,30.00650651,2019-05-23T02:22:58.043698Z,3.7,NaN,NaN,NaN
S1048d,mqs2021vwbn,Elysium Southeast,2021-11-07T22:00:15.254098Z,A,100,30.18618619,2021-11-07T22:04:04.40886Z,3.6,NaN,NaN,NaN
S1022a,mqs2021tyvj,Elysium Northeast,2021-10-11T23:14:29.105382Z,A,63,30.72522523,2021-10-11T23:18:20.004721Z,3.6,NaN,NaN,NaN
S0820a,mqs2021fjzq,Aeolis Northeast,2021-03-18T14:51:33.869889Z,A,88,30.18618619,2021-03-18T14:54:19.835741Z,3.3,NaN,NaN,NaN
